In [ ]:
!kaggle datasets download -d stanislavko/hwnas-dataset-music-mood-classification -p /content/data


Dataset URL: https://www.kaggle.com/datasets/stanislavko/hwnas-dataset-music-mood-classification
License(s): other
hwnas-dataset-music-mood-classification.zip: Skipping, found more recently modified local copy (use --force to force download)


In [6]:
# ============================================================
# STEP 0: Install + Imports
# ============================================================
!pip install librosa kaggle pandas scikit-learn

import os, librosa, numpy as np, pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

# ============================================================
# STEP 1: Download Dataset from Kaggle (requires API key)
# ============================================================
# Upload kaggle.json from your Kaggle account (Account > API > Create New API Token)
from google.colab import files
files.upload()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

# Download dataset
!kaggle datasets download -d stanislavko/hwnas-dataset-music-mood-classification -p /content/data
!unzip -q /content/data/hwnas-dataset-music-mood-classification.zip -d /content/data

# ============================================================
# STEP 2: Feature extraction function
# ============================================================
def extract_librosa_features(file_path):
    try:
        y, sr = librosa.load(file_path, duration=60)
        tempo, _ = librosa.beat.beat_track(y=y, sr=sr)

        features = {
            "tempo": float(tempo),
            "rms": float(np.mean(librosa.feature.rms(y=y))),
            "spectral_centroid": float(np.mean(librosa.feature.spectral_centroid(y=y, sr=sr))),
            "spectral_bandwidth": float(np.mean(librosa.feature.spectral_bandwidth(y=y, sr=sr))),
            "zero_crossing": float(np.mean(librosa.feature.zero_crossing_rate(y))),
            "mfcc1": float(np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)[0])),
            "mfcc2": float(np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13)[1]))
        }
        return features
    except Exception as e:
        print("⚠️ Error with", file_path, ":", e)
        return None

# ============================================================
# STEP 3: Build feature dataset
# ============================================================
data_folder = "/content/data/Data"
moods = []
features_list = []

for mood_dir in os.listdir(data_folder):
    mood_path = os.path.join(data_folder, mood_dir)
    if os.path.isdir(mood_path):
        for file in os.listdir(mood_path):
            if file.endswith((".mp3",".wav")):
                path = os.path.join(mood_path, file)
                feats = extract_librosa_features(path)
                if feats:
                    feats["mood"] = mood_dir
                    features_list.append(feats)

df_features = pd.DataFrame(features_list)
print("✅ Feature dataset shape:", df_features.shape)
df_features.head()

# ============================================================
# STEP 4: Train/Test Split & Train Random Forest
# ============================================================
X = df_features.drop("mood", axis=1)
y = df_features["mood"]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

rf = RandomForestClassifier(n_estimators=200, random_state=42, class_weight="balanced")
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)
print("✅ Accuracy:", accuracy_score(y_test, y_pred))
print(classification_report(y_test, y_pred))

# ============================================================
# STEP 5: Predict mood for uploaded song
# ============================================================
from google.colab import files
uploaded = files.upload()

for fn in uploaded.keys():
    feats = extract_librosa_features(fn)
    if feats:
        X_new = pd.DataFrame([feats])
        mood = rf.predict(X_new)[0]
        print(f"🎶 {fn} → Predicted Mood: {mood}")

y


Saving kaggle.json to kaggle (3).json
Dataset URL: https://www.kaggle.com/datasets/stanislavko/hwnas-dataset-music-mood-classification
License(s): other
hwnas-dataset-music-mood-classification.zip: Skipping, found more recently modified local copy (use --force to force download)
replace /content/data/Data/angry/angry_00000.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/data/Data/angry/angry_00001.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/data/Data/angry/angry_00002.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/data/Data/angry/angry_00003.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/data/Data/angry/angry_00004.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/data/Data/dark/dark_00000.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/data/Data/dark/dark_00001.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replace /content/data/Data/dark/dark_00002.wav? [y]es, [n]o, [A]ll, [N]one, [r]ename: y
replac

/tmp/ipython-input-276114966.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  "tempo": float(tempo),


✅ Feature dataset shape: (70, 8)
✅ Accuracy: 0.2857142857142857
              precision    recall  f1-score   support

       angry       1.00      1.00      1.00         1
        dark       0.00      0.00      0.00         1
   energetic       0.00      0.00      0.00         1
        epic       0.00      0.00      0.00         1
    euphoric       0.00      0.00      0.00         1
   glamorous       0.00      0.00      0.00         1
       happy       0.00      0.00      0.00         1
  mysterious       0.50      1.00      0.67         1
    relaxing       0.50      1.00      0.67         1
    romantic       0.00      0.00      0.00         1
         sad       0.00      0.00      0.00         1
       scary       0.00      0.00      0.00         1
 sentimental       0.33      1.00      0.50         1
   uplifting       0.00      0.00      0.00         1

    accuracy                           0.29        14
   macro avg       0.17      0.29      0.20        14
weighted avg    

/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.12/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


Saving Anthony Vega - Lions Dance.mp3 to Anthony Vega - Lions Dance.mp3


/tmp/ipython-input-276114966.py:35: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  "tempo": float(tempo),


🎶 Anthony Vega - Lions Dance.mp3 → Predicted Mood: energetic
